## Кривоногов Н.В., INN, практическое задание № 2

Используем набор примеров fashion-MNIST
1. Опишите - какой результат нейросети получен в зависимости от:
  - числа нейронов в слое (для 2-х слойной сети), 
  - числа слоев (2, 3, 5, 10) при близких размерах сети (близкое число тренируемых параметров).
  - фиксируйте для тренировочного и тестового набора метрики accuracy.
2.  Проверьте работу разных оптимизаторов (SGD, Adam, RMSProp) для одной из моделей п.1. Фиксируйте для тренировочного и тестового набора метрики accuracy.

3. Сделайте вывод - что помогло вам улучшить качество классификации в нейросети на тестовом наборе? 

4. Для одного варианта сети сформируйте матрицу ошибок по классам. Оцените качество модели по каждому классу отдельно (полнота, точность).

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import keras
from keras.datasets import fashion_mnist

In [2]:
(X_train, y_train_labels), (X_test, y_test_labels) = fashion_mnist.load_data()

In [3]:
# Нормировка
X_train = X_train / 255.0
X_test = X_test / 255.0

In [4]:
# Вытягивание признаков
X_train = X_train.reshape((-1, 28 * 28))
X_test = X_test.reshape((-1, 28 * 28))

In [5]:
from keras.layers import Dense       # Полносвязный линейный слой
from keras.models import Sequential  # Класс последовательности слоев

In [6]:
num_neurons_1 = 64 # 32, 16, 8
num_neurons_2 = 10

model = Sequential()
model.add(Dense(num_neurons_1, input_shape=(28 * 28,), activation='relu'))
model.add(Dense(num_neurons_2, activation='sigmoid'))

model.summary()

In [7]:
# пример на лекции показал, что лучшим оптимизатором является Adam: 

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [8]:
from keras.utils.np_utils import to_categorical

y_train = to_categorical(y_train_labels)
y_test = to_categorical(y_test_labels)

In [9]:
# передача обучающего датасета keras модели

num_epochs = 4
batch_size = 32

history = model.fit(X_train, y_train,
                    epochs=num_epochs,
                    batch_size=batch_size,
                    validation_data=(X_test, y_test),
                    verbose=1)

- Число нейронов в слое (для 2-х слойной сети) = 64 и 10, метрики на train = 0.8786, на test = 0.8626.

- Число нейронов в слое (для 2-х слойной сети) = 32 и 10, метрики на train = 0.8690, на test = 0.8552.

- Число нейронов в слое (для 2-х слойной сети) = 16 и 10, метрики на train = 0.8625, на test = 0.8491.

- Число нейронов в слое (для 2-х слойной сети) = 8 и 10, метрики на train = 0.8440, на test = 0.8305.

Вывод 1: при увеличении количества нейронов в слое метрика возрастает (и наоборот соответственно). 

In [10]:
model = Sequential()
model.add(Dense(16, input_shape=(28 * 28,), activation='relu'))
model.add(Dense(16, input_shape=(28 * 28,), activation='relu'))
model.add(Dense(16, input_shape=(28 * 28,), activation='relu'))
model.add(Dense(16, input_shape=(28 * 28,), activation='relu'))
# model.add(Dense(16, input_shape=(28 * 28,), activation='relu'))
# model.add(Dense(16, input_shape=(28 * 28,), activation='relu'))
# model.add(Dense(16, input_shape=(28 * 28,), activation='relu'))
# model.add(Dense(16, input_shape=(28 * 28,), activation='relu'))
# model.add(Dense(16, input_shape=(28 * 28,), activation='relu'))
model.add(Dense(10, activation='sigmoid'))

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

num_epochs = 4
batch_size = 32

history = model.fit(X_train, y_train,
                    epochs=num_epochs,
                    batch_size=batch_size,
                    validation_data=(X_test, y_test),
                    verbose=1)

- Число слоев = 2, метрики на train = 0.8613, на test = 0.8466.

- Число слоев = 3, метрики на train = 0.8629, на test = 0.8572.

- Число слоев = 5, метрики на train = 0.8560, на test = 0.8461.

- Число слоев = 10, метрики на train = 0.8503, на test = 0.8365.

Вывод 2: зависимость метрики от числа слоев имеет нелинейный характер. Так в данном примере метрика максимальна при числе слоев, равном трем. 

In [11]:
%%time

# буду сохранять модели в словарь:
models = {}

plt.figure(figsize=(16, 7))
colors = ['g', 'b', 'r', 'y']

num_epochs = 4
batch_size = 32

# вектор для вывода результатов:
epoch = np.arange(num_epochs+1)

# буду изменять оптимизаторы:
for i, i_optim in enumerate([keras.optimizers.SGD(),
                             keras.optimizers.Adam(),
                             keras.optimizers.RMSprop()]):
    
    # создаю рабочую модель model_i, куда буду загружать эти веса (она должна быть идентичной структуры):
    num_neurons_1 = 64
    num_neurons_2 = 10

    model_i = Sequential()
    model_i.add(Dense(num_neurons_1, input_shape=(28 * 28,), activation='relu'))
    model_i.add(Dense(num_neurons_2, activation='sigmoid'))
    
    # компилирую model_i с одним из оптимизаторов:
    model_i.compile(
        optimizer=i_optim,
        loss='categorical_crossentropy',
        metrics=['accuracy'],
        )
    
    # вычисляю ошибку для model_i без обучения:
    h0_train = model_i.evaluate(X_train, y_train, verbose=0)
    h0_val = model_i.evaluate(X_test, y_test, verbose=0)

    # провожу обучение модели:
    h = model_i.fit(X_train, y_train,
                    epochs=num_epochs,
                    batch_size=batch_size,
                    validation_data=(X_test, y_test),
                    verbose=1)
    
    # записываю обученную модель в словарь:
    models[i_optim.get_config()['name']] = model_i

    # строю графики:
    plt.plot(epoch, np.log([h0_train[0]] + h.history['loss']),
             '-', c=colors[i],
             label=model_i.optimizer.get_config()['name'] + ' train')
    plt.plot(epoch, np.log([h0_val[0]] + h.history['val_loss']),
             '--', c=colors[i],
             label=model_i.optimizer.get_config()['name'] + ' val')
    print('=' * 20)

plt.legend()
plt.title('нейросети от оптимизатора log(loss)')
plt.xlabel('epoch')
plt.ylabel('log(loss)')
plt.show()

- Оптимизатор SGD, метрики на train = 0.8446, на test = 0.8352.

- Оптимизатор Adam, метрики на train = 0.8810, на test = 0.8576.

- Оптимизатор RMSProp, метрики на train = 0.8802, на test = 0.8511.

Вывод 3: оптимизатор Adam - лучший, как на train, так и на test.

Вывод 4: качество классификации в нейросети на тестовом наборе помогает улучшить подбор её оптимальных параметров, таких как: количество нейронов в слое, количество слоев, оптимизатор. 

In [12]:
model = Sequential()
model.add(Dense(32, input_shape=(28 * 28,), activation='relu'))
model.add(Dense(32, input_shape=(28 * 28,), activation='relu'))
model.add(Dense(10, activation='sigmoid'))

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

num_epochs = 4
batch_size = 32

history = model.fit(X_train, y_train,
                    epochs=num_epochs,
                    batch_size=batch_size,
                    validation_data=(X_test, y_test),
                    verbose=1)

Epoch 1/4
1875/1875 [==============================] - 8s 3ms/step - loss: 0.5438 - accuracy: 0.8104 - val_loss: 0.4495 - val_accuracy: 0.8422
Epoch 2/4
1875/1875 [==============================] - 6s 3ms/step - loss: 0.4013 - accuracy: 0.8561 - val_loss: 0.4119 - val_accuracy: 0.8544
Epoch 3/4
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3632 - accuracy: 0.8676 - val_loss: 0.4002 - val_accuracy: 0.8539
Epoch 4/4
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3422 - accuracy: 0.8760 - val_loss: 0.4185 - val_accuracy: 0.8437


In [13]:
y_pred = model.predict(X_test)

313/313 [==============================] - 1s 2ms/step


In [14]:
y_pred_class = np.argmax(y_pred, axis=1)

In [15]:
y_test_class = np.argmax(y_test, axis=1)

In [16]:
from sklearn.metrics import confusion_matrix

In [18]:
# формирую матрицу ошибок по классам:

print(confusion_matrix(y_test_class, y_pred_class))

[[880   1  18  22   1   0  61   0  17   0]
 [  7 959   2  27   2   0   1   0   2   0]
 [ 15   1 905  11  25   0  37   0   6   0]
 [ 48   7  13 889  21   0  18   0   4   0]
 [  1   0 411  50 466   0  69   0   3   0]
 [  0   0   0   1   0 938   0  35   2  24]
 [195   1 193  26  37   1 531   0  16   0]
 [  0   0   0   0   0  25   0 937   0  38]
 [  6   1   6   5   0   2   4   3 973   0]
 [  0   0   0   0   0   4   1  36   0 959]]


In [19]:
from sklearn.metrics import classification_report

In [23]:
# качество модели по каждому классу отдельно (полнота, точность):

print(classification_report(y_test_class, y_pred_class))

              precision    recall  f1-score   support

           0       0.76      0.88      0.82      1000
           1       0.99      0.96      0.97      1000
           2       0.58      0.91      0.71      1000
           3       0.86      0.89      0.88      1000
           4       0.84      0.47      0.60      1000
           5       0.97      0.94      0.95      1000
           6       0.74      0.53      0.62      1000
           7       0.93      0.94      0.93      1000
           8       0.95      0.97      0.96      1000
           9       0.94      0.96      0.95      1000

    accuracy                           0.84     10000
   macro avg       0.86      0.84      0.84     10000
weighted avg       0.86      0.84      0.84     10000



Вывод 5: для класса "1" точность максимальна, для класса "6" точность минимальна; для класса "8" полнота максимальна, для класса "4" полнота минимальна. 